In [37]:
import urllib.request
from bs4 import BeautifulSoup
import re
import time
import sqlite3
from random import randint


In [38]:
def get_review(album, artist):
    con = sqlite3.connect('reviews.db')
    cur = con.cursor()
    cur.execute("SELECT * FROM reviews WHERE album=? AND artist=?", (album, artist))
    return cur.fetchone()

In [39]:
def insert_review(album, artist, rating, release_year, genre, review_date):
    con = sqlite3.connect("reviews.db")
    cur = con.cursor()
    cur.execute("""
    INSERT INTO reviews (album, artist, rating, release_year, genre, review_date) VALUES (?, ?, ?, ?, ?, ?)
    """, (album, artist, rating, release_year, genre, review_date))
    con.commit()

In [40]:
# Get links to reviews
def parse_reviews(page):
    with urllib.request.urlopen('https://pitchfork.com/reviews/albums/?page=' + str(page)) as f:
        soup = BeautifulSoup(f)
        links = soup.find_all('a', class_='review__link')
        for link in links:
            url = 'https://pitchfork.com' + link.get('href')
            with urllib.request.urlopen(url) as f:
                review = BeautifulSoup(f)
                album = review.body.main.article.header.find('h1').text
                artist = review.body.main.article.header.find('ul').text
                rating = review.body.main.article.header.find('div', class_=re.compile("^SplitScreenContentHeaderScoreBox")).text
                release_year = review.body.main.article.header.find('time', class_=re.compile("^SplitScreenContentHeaderReleaseYear")).text

                info_items = review.body.main.article.header.find_all('div', class_=re.compile("^InfoSliceItem"))
                genre = info_items[0].text.split(":")[1].strip()
                review_date = info_items[2].text.split(":")[1].strip()
                print(album, "--", artist, "--", rating, "--", release_year, "--", genre, "--", review_date)

                if get_review(album, artist) is None:
                    print("> Inserting: ", album, "|", artist, "|", rating)
                    insert_review(album, artist, rating, release_year, genre, review_date)
                time.sleep(randint(5,18))

In [41]:
x = range(1, 15)
for n in x:
    print("Parsing Reviews on page", n)
    parse_reviews(n)
    wait = randint(60,300)
    print("...Waiting", wait, "seconds")
    time.sleep(wait)


Parsing Reviews on page 1
Crying Laughing Loving Lying -- Labi Siffre -- 9.0 -- 1972 -- Pop/R&B / Folk/Country -- June 11, 2023
Metro Boomin Presents Spider-Man: Across the Spider-Verse (Soundtrack From and Inspired by the Motion Picture) -- Metro Boomin -- 6.4 -- 2023 -- Rap -- June 10, 2023
The Age of Pleasure -- Janelle Monáe -- 7.6 -- 2023 -- Pop/R&B -- June 9, 2023
Space Heavy -- King Krule -- 7.5 -- 2023 -- Rock -- June 9, 2023
O Monolith -- Squid -- 7.3 -- 2023 -- Rock -- June 9, 2023
Weathervanes -- Jason Isbell and the 400 Unit -- 7.5 -- 2023 -- Folk/Country -- June 8, 2023
New Jazz -- Lunchbox -- 7.5 -- 2023 -- Rap -- June 8, 2023
QWERTY EP -- Saya Gray -- 7.0 -- 2023 -- Pop/R&B -- June 8, 2023
Wandering Rebel -- Juan Wauters -- 7.7 -- 2023 -- Rock -- June 7, 2023
Jam City Presents EFM -- Jam City -- 7.4 -- 2023 -- Electronic -- June 7, 2023
Zango -- Witch -- 7.7 -- 2023 -- Rock -- June 7, 2023
> Inserting:  Zango -- Witch -- 7.7
Orbs -- Anthony Naples -- 8.0 -- 2023 -- Elect